# Test Model

In this notebook, we are testing our model performance on test Dataset.

**1. Import Required Libraries:-** 

In [1]:
from torchvision import transforms
import torch
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import time
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
#Local Imports
from transforms import Normalize,ToTensor
from dataset import HeadposeDataset
from model import FSANet

%matplotlib inline

**2. Compose Augmentation Transform and Create Dataset:-**

In [2]:
transform = transforms.Compose([
            Normalize(mean=127.5,std=128),
            ToTensor()
            ])

data_path = '../data/type1/test'

hdb = HeadposeDataset(data_path,transform=transform)

x (images) shape:  (1969, 64, 64, 3)
y (poses) shape:  (1969, 3)


**3. Create Dataloader to batch over test dataset:-**

In [3]:
#Setup dataloaders for train and validation
batch_size = 64

test_loader = DataLoader(hdb, 
                          batch_size=batch_size,
                          shuffle=False)

**4. Define Model Function:-**

In [4]:
model = FSANet(var=False)
#Load Model Checkpoint
chkpt_dic = torch.load('../checkpoints/fsa1x1-08082020.chkpt')
model.load_state_dict(chkpt_dic['best_states']['model'])

<All keys matched successfully>

**5. Place Model in GPU:-**

In [5]:
#Place all the necessary things in GPU 
device = torch.device("cuda")
model.to(device)

FSANet(
  (msms): MultiStreamMultiStage(
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (s0_conv0): SepConvBlock(
      (conv): SepConv2d(
        (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
        (pointwise): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
    (s0_conv1_0): SepConvBlock(
      (conv): SepConv2d(
        (depthwise): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (pointwise): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
    (s0_conv1_1): SepConvBlock(
      (conv): SepConv2d(
        (depthwise): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (pointwise): Conv2d(32, 32, kernel_si

**6. Define Testing Function:-**

In [6]:
def test_net():
    yaw_loss = []
    pitch_loss = []
    roll_loss = []
    model.eval()
    with torch.no_grad():
        for batch_i, data in enumerate(test_loader):
            # get the input images and their corresponding poses
            images,gt_poses = data

            # put data inside gpu
            images = images.float().to(device)
            gt_poses = gt_poses.float().to(device)

            # call model forward pass
            predicted_poses = model(images)

            abs_loss = torch.abs(gt_poses-predicted_poses)
            
            abs_loss = abs_loss.cpu().numpy().mean(axis=0)
            
            yaw_loss.append(abs_loss[0])
            pitch_loss.append(abs_loss[1])
            roll_loss.append(abs_loss[2])
    
    yaw_loss = np.mean(yaw_loss)
    pitch_loss = np.mean(pitch_loss)   
    roll_loss = np.mean(roll_loss)
    print('Mean Absolute Error:-')
    print(f'Yaw: {yaw_loss:.2f}, Pitch: {pitch_loss:.2f}, Roll: {roll_loss:.2f}')

**7. Test Model and print Mean Absolute Error:-**

In [7]:
try:
    # test your network
    test_net()
except KeyboardInterrupt:
    print('Stopping Testing...')

Mean Absolute Error:-
Yaw: 4.85, Pitch: 6.27, Roll: 4.96
